In [ ]:
import os
from ansys.acp.core import launch_acp, Client

In [ ]:
SERVER_BIN_PATH = "C:/ANSYSDev/acp/bin/release_with_debug_info/lib/acp_grpcserver.exe"

In [ ]:
server = launch_acp(SERVER_BIN_PATH)

In [ ]:
client = Client(server=server)

In [ ]:
# TODO: infer from file ending
model = client.import_model(path=os.path.abspath("./class40.cdb"), format="ansys:cdb")

In [ ]:
model.update()

In [ ]:
from ansys.api.acp.v0 import model_pb2, model_pb2_grpc

In [ ]:
model_stub = model._get_stub()

In [ ]:
request = model_pb2.SaveAnalysisModelRequest(resource_path=model._resource_path, path=os.path.abspath("./results/class40_analysis_model.cdb"))

In [ ]:
model_stub.SaveAnalysisModel(request)

In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl


In [ ]:
mapdl = launch_mapdl(additional_switches="-smp")

In [ ]:
mapdl.input("class40_analysis_model.cdb")

In [ ]:
mapdl.allsel()
mapdl.slashsolu()
mapdl.solve()

In [ ]:
mapdl.post1()
mapdl.set("last")
mapdl.post_processing.plot_nodal_displacement(component="NORM")

In [ ]:
mapdl.post_processing.plot_nodal_displacement(component="NORM")